In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import os
if torch.cuda.is_available(): device = 'cuda'
else: device = 'cpu'

In [2]:
# open text file and read in data as `text`
filename = 'bai_van_mau.txt'
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])
# encoded = np.array([ch for ch in text])

In [3]:
def pre_process(arr, batch_size, seq_length):
        # start at random location within seg_length
    start = int(np.random.choice(range(seq_length),1))
    arr = arr[start:]
    
    # total number of batches we can make, // integer division, round down

    total_seq_size = len(arr)//(batch_size)
    # Keep only enough characters to make full batches
    arr = arr[:total_seq_size*batch_size]
    
    # Reshape into batch_size rows, n. of first row is the batch size, the other lenght is inferred
    arr = arr.reshape((batch_size, -1))
    arr = arr[:,:((total_seq_size)//seq_length)*seq_length + 1]
    
    X = []
    Y = []
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1]-1, seq_length):
        # The features

        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = arr[:, n+1:n+seq_length+1]
        # TODO: investigate
        X.append(x)
        Y.append(y)
    
    return torch.as_tensor(X), torch.as_tensor(Y)
a = np.random.randn(435435)
x,y = pre_process(a, 100,200)
x.shape, y.shape


(torch.Size([21, 100, 200]), torch.Size([21, 100, 200]))

In [4]:
encoded[:100]

array([180, 102, 217, 235, 119, 114, 102, 188,  94,  69, 102,  89, 186,
        69, 171, 102, 143, 196,  67, 204, 153, 102, 180, 102, 184, 204,
       204, 204, 166, 102, 173, 167, 102, 114, 176, 118, 102,  69, 235,
       167, 102,  22, 177,  22, 235, 102, 114, 119,  69, 171, 102,  69,
       236,  59, 102, 118,  59,   8,  69, 171, 111, 102, 114, 176, 118,
       102,  69, 235, 167, 102,  92,  94,  69, 102, 235,  35, 230, 102,
       173,  53,  69, 102,  22, 163, 230, 102,  44, 158,  69, 102, 118,
       176,  22, 197,  61, 180, 102, 133,  94, 114])

In [5]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

def save_checkpoint(file_name):
    model_dante = file_name+'.net'

    checkpoint = {'n_hidden': net.n_hidden,
                  'n_layers': net.n_layers,
                  'state_dict': net.state_dict(),
                  'tokens': net.chars}

    with open(model_dante, 'wb') as f:
        torch.save(checkpoint, f)

In [6]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [7]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=612, n_layers=4,
                               drop_prob=0.5, lr=0.001, bidirectional=False):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        if bidirectional: self.D = 2
        else: self.D = 1
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # define encoder
        self.embedding = nn.Embedding(len(self.chars), 300)
        
        ## TODO: define the LSTM
#         self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
#                             dropout=drop_prob, batch_first=True)
        self.lstm = nn.LSTM(300, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True, bidirectional=bidirectional)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(self.D*n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        
        word_embedded = self.embedding(x)        
        ## TODO: Get the outputs and the new hidden state from the lstm
#         r_output, hidden = self.lstm(x, hidden)
        r_output, hidden = self.lstm(word_embedded, hidden)
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.D*self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
       
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
#         weight = next(self.parameters()).data
       
        # TODO: investigate
        hidden = (torch.zeros((self.D*self.n_layers, batch_size, self.n_hidden), device=device),
              torch.zeros((self.D*self.n_layers, batch_size, self.n_hidden), device=device))
        
        return hidden

In [8]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
#         x = one_hot_encode(x, len(net.chars))

        inputs = torch.as_tensor(x).to(device)
        

        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        # apply softmax to get p probabilities for the likely next character giving x
        p = F.softmax(out, dim=1).data
        p = p.cpu() # move to cpu
        
        # get top characters
        # considering the k most probable characters with topk method
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [9]:
def sample(net, size, prime='Cháu lên ba ', top_k=None):
        


    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)
    return ''.join(chars)

In [10]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10, checkpoint='data.net'):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''

    net.train() # set training mode for dropout and batchnorm
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss().to(device)
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    train_data, val_data = data[:val_idx], data[val_idx:]
    
    h = net.init_hidden(batch_size)
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
                    
        counter = 0
        step = len(data)//(batch_size*seq_length)
        X,Y = pre_process(train_data, batch_size, seq_length)
        for inputs, targets in zip(X,Y):
            inputs, targets = inputs.to(device), targets.to(device)
            counter += 1
            

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h]) # similar to detach

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                net.eval()
                
                val_h = net.init_hidden(batch_size)
                val_losses = 0
                X_val,Y_val = pre_process(val_data, batch_size, seq_length)
                for inputs, targets in zip(X_val,Y_val):
                    inputs, targets = inputs.to(device), targets.to(device)
                
                    val_h = tuple([each.data for each in val_h])

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                    val_losses = 0.9*val_losses +0.1*val_loss.item()

                

                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}/{}...".format(counter,step),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(val_losses))
                print(sample(net, 1000, prime='Thế nào ', top_k=5))
                net.train() # reset to train mode after iterationg through validation data
                save_checkpoint(checkpoint)
            

In [11]:
# define and print the net
n_hidden=512
n_layers=4

net = CharRNN(chars, n_hidden, n_layers)
net.to(device)

CharRNN(
  (embedding): Embedding(244, 300)
  (lstm): LSTM(300, 512, num_layers=4, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=244, bias=True)
)

In [ ]:
batch_size = 128
seq_length = 200 #max length verses
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=300, checkpoint=filename)

Epoch: 1/50... Step: 300/771... Loss: 2.0895... Val Loss: 2.0148
Thế nào ngoang nrười viển chú với những và manh những những đàn những tónh cáp tói trường như đàn chúng chi việc chanh cáng.
Những và mà như tóc chi thoơi tói, tôc, chanh trong với tiynh đàn vại như điểt đành của đàn, điết tranh của ngàn,, điền thoy trai vạn các thiến và ngoya nhữi thoờng nhữi vàn, trang, viềt đàm điệc có nhà chu như đài tho thược tràng đi trong đại người cho tràng trườm chúng điển troi nhà nghơ mi trong thanh như mà cho trong, đài ngữy của như những đại như mộn thường thường trường đà mà thoya chuan. Thiếng, trười cruau nrữ đi cho thướng thiếc thường vang ngương tho nhữi thoời ngữ vộnh, ngoya chiện trong trười thoyệu cáng người đanh. Trang đi crong chanh những chiết chonh thúc thoac thoyện điệng động tôi điệu và những của nhông chiệc trong của thiếc ngà như của thương thúi thoya của cát tráng ngoài niêm điệc đội đi thàng đi thưởi chang, thưởng nhữi cáng cho đàn chong chiêc đà cho những và, nhữnh nihn đàu

Epoch: 5/50... Step: 300/771... Loss: 1.4876... Val Loss: 1.3347
Thế nào chỉ đã trở thành một con người thương trí để tình yêu để làm thư trước những tình yêu thương, cho cô gần, những chiếc ta như những người thế giới thiệu được cách sống, nói và chỉ chăm chỉ để thể hiện các nón những năm nghĩ với cách hoạt thức, những trong số ngày thơ và cho con người trong người dưới thương những lòng thương nghĩa và tự tin, tinh tương đầy đi vinh thanh được thời gian trên đời tham gia của các bác Việt Nam.
Ngày 19 ngoài đường đã là thế giới với chính độc độ trở thành tiến và nhà trường và con sáng người.
- Thiên nhiên của người ta đã thấy thương chiến cho mình. Mùi thơm thật đẹp và được mẹ thư và trong, ngoài ngày các cá cho ta đã thể hiện chuyên trốn cho con người thiếu thực vào mình trong những bài văn nghệ sĩ: "Thế này của cô giáo như trong văn mật của con người còn có nghiêm thanh và tiếng nhân để nghĩ đến thế giới để thành người cho các tình yêu, được để nhận ra nhau có một những trái hoang l

Epoch: 9/50... Step: 300/771... Loss: 1.3585... Val Loss: 1.2282
Thế nào để trở thành truyền thống để trong mình cách có những bài thơ của con người. Trong cuộc sống trước những cái tôi được điều đó, con người thầm thương trong mọi người, đã thấy chú đi học của người đọc.
Các bạn thân thiết trong làng của cô thể hiện nhiều cảm xúc. Câu chuyện nhiệt lục trong lớp của em, những đoạn hoà của bà tôi được bố mẹ chỉ làm thấy được một nghề nghe nhà nước ngày nào. Trông có những con đi làn dưới mặt trời, tôi đang chỉ là cái trống tôi vẫn thấy chúng tôi đã có đi vào bài thơ.
Bạn có mỗi khi biết đây tôi nhớ chú bác trưởng thành của mình, thương em vẫn luôn nghỉ học với bạn trên.
Em có thể điều khiển nhiệm vụ nhiều người này, trong cuộc sống của thư đã từng được tận hưởng một công việc đó cũng là một con số vật dịch. Chúng ta chỉ thật cảm thấy được tin cho những điều chúc mình và trong chúng ta, cả một người bà cũng luôn có những người đã có thấy những chú gà nó có được trở nên đẹp. Chính như một

Epoch: 13/50... Step: 300/771... Loss: 1.3149... Val Loss: 1.1802
Thế nào của mình thì đến ngày trước đây nhà thơ này thôi, để lại thành tích này và đi đến năm 1400, trong ngày trước thần tiên, thuộc làng Hùng, nhưng chúng tôi đều chia sẻ, cách đi thì chúng tôi lại trong đó chúng ta đã từng được ba mẹ đến trường.
Trong làng bố của em cho con người đã đi được thứ nhất đó là nguồn gốc của nước.
Trong các trò chơi đã có nhiều cái đẹp, chiến đấu, thay đổi, được cái đẹp đã có một cô gái trong lòng mình, trường em có thể tự học có thể trở thành một bài văn của tuổi thơ của mình. Có những con người đang tìm câu thơ và chúng tôi nhớ, cô lớp mẹ ngoại được chia lại về nhân vật đó là những người bạn thân mới nghĩa quý với mình. Một chuyện của chúng tôi đã được nhiều lần có thể nghiêm túc nhưng lại thấy con tiếng chim nghe điểm mà con người chúng ta thì nó cũng có người đọc trở thành người trong cuộc đời.
Những bài văn có nghĩa là một người có nhận định trên tình yêu thương trong trẻo và như thế.


Epoch: 17/50... Step: 300/771... Loss: 1.2811... Val Loss: 1.1528
Thế nào cũng là cái cô thương chiếc thuyền nhất, chỉ to như chim bồ câu, chuẩn bị nguồn nhưng những ngày này cũng có thể là nhân tính trong nhà với mỗi thời gian:
Trong những tình cảm của em luôn thư như có lẽ là một câu chuyện về nhiều bài học thiết thực. Năm nay: tiếp theo đó là bài viết dành cho em, những câu chuyện đầy tình trạng trước cách mạng tháng tám với em. Em thấy mỗi khi em được chiến thắng của mọi cô chú. Chúng tôi nhớ thấy được một cô giáo của mình. Có lẽ trong lớp mình thấy mẹ đang làm từ đó cho em về cuộc sống nghiêng nhất, trông rất thú vị. Thế là nó trôi qua, cả một ngày thường thấy mẹ thường nghe mỗi khi cô lại đã thấy thấy thương mẹ của cô gắng học hỏi, nhớ những công việc tốt của các em chăm bà.
Chúng em trong công việc của em đến trường, cô đều đều từ trước cho mọi người có những thế hệ này đã trở thành nhiều bạn bè và cả nhà từ nhỏ. Cô là chiếc đồng hồ và thầy luôn được trả lời cho tôi và mình nhiề

Epoch: 21/50... Step: 300/771... Loss: 1.2584... Val Loss: 1.1336
Thế nào đó là những người con nhỏ của mình. Có những ngày cuối tuần, chỉ có những buổi trưa hè đã là tình thương nghịch ngợm trên tất cả những người bạn hay nhất. Nhưng có thể thấy mình vừa thương cho chúng tôi, người mẹ còn ngày càng thay đổi, trong xanh của nhân dịp sau, để trở lại, nón được trở thành người mà mọi người thấy trong lòng chúng tôi nghĩ về thành phố. Chẳng còn đi ngủ chuyện về mình đã giúp bạn chơi với các bạn học sinh chúng tôi trở thành một người có thể nào được thanh thản và cầm chiếc bút
Trên trái đất của cậu có cảm giác trường trung dương được nghe chuyện nghiên cứu.
Trong công việc nào cũng tin rằng, các bạn có thể được coi là những biểu hiện chu đáo và tinh khôi. Chúng em nhận thấy một thầy giáo của tôi là nơi có nhiều đứa trẻ đã trở thành một bài học cao đẹp, chúng ta sẽ không thể nào quên tình trạng trường thi của mình thì chỉ là một chuyến đi của bạn bè, thầy cô và điều đó, tôi còn có những truy

Epoch: 25/50... Step: 300/771... Loss: 1.2329... Val Loss: 1.1211
Thế nào chính là một người bạn nào đó vì thấy tình cảm của mình. Có như vậy, tôi luôn được người có cái ngoài cá nhân và để làm trò được chung thiết nghiêm và chia sẻ cả những người bạn của mình và thấy đôi bàn tay được biết đến. Nhưng nhà thơ đã giúp người đọc có một ngày mới trong tình yêu quê, đó là chuyện để được trở thành những người bạn hiền lành trong tâm hồn con người.
Bàn tay người con gái của cô giáo, tôi luôn nghe những trò chơi, cảm ơn bà của mình thì mẹ nói chuyện với chúng em trên bờ thời gian này.
Từ ngày, chiếc điện thoại có cái trang phục nhưng điều khiển đi chơi đầy sự hàng ngày trước người đã trôi qua, có thể nói chuyện về chúng tôi. Cô giáo cũng có thể chúng tôi thấy đôi bàn tay con của tôi. Tôi đã làm được thành công của mình và cảnh vật vì thường được tôi chăm chỉ luôn có một buổi sáng người mình. Cậu bé cho tôi thấy cô giáo trưởng thành của mình nghiên phắc để trở nên có thêm những chú trẻ của mình

Epoch: 29/50... Step: 300/771... Loss: 1.2249... Val Loss: 1.1117
Thế nào của mình đã làm cho thầy giáo thì lúc nào cũng chăm chỉ, nuôi mẹ, nhưng có lẽ chúng tôi được trả lại cho các em như chúc cháu. Nó không chỉ là người bạn tốt chính là bài học của mình. Những nỗi đau của cô giáo có thể làm nghề thật đáng yêu nhưng chúng tôi chúng tôi lại cảm thấy nhìn chúng tôi và mẹ cũng trở nên thật nhiều, ngoại của cậu nhé!
Mẹ có một bộ đồ cho mẹ với nhau. Nhưng ngày hôm sau nhà em thích nó cũng như chiếc đồng hồ chúng em tranh nhau được bố chăm sóc cô trong bài tập cùng chúng tôi chăm sóc bà nội em, thầy đã lớn, mẹ tôi đều thường đi chơi để nó vào lớp, nhớ đến những chú chó tôi chợt nhớ đến thầy cô. Nó không còn trong lòng tôi. Nói chuyện nên cô đều đã có một cô gái nhẹ nhàng cảnh trong nhà và các tôi đã thấy người ta có thể nghiêm khắc, mẹ đã nuôi một con trâu của nó, chẳng ngủ, trong chính những ngày thanh niên chính là cô bé đã có những con đường đến trường tôi ngoài đường nói lên cả một cô 

Epoch: 33/50... Step: 300/771... Loss: 1.2063... Val Loss: 1.1027
Thế nào có cái cả một người trẻ của tôi có một con người tham gia tinh tế.
Nói đến cuộc đời của tôi. Ngoài cái tôi, nàng thường ngồi nhìn thấy những tiếng gọi của chị thanh mát. Chú còn có chiếc tủ làm một chiếc điều. Tôi luôn có những người bạn đưa đất trời trước những người chị thường trong chiếc trống cho con cái. Mỗi khi cô được tôi chỉ chỉ cần bay trên những bông lúa thật đặc biệt, có mẹ có thể là cha mẹ tôi cũng thấy được ngày nào đây cô đi làm được thế nào? Thế là con còn lại chúng tôi chỉ thấy điều đó, cô luôn thấy những người cô giảng bài trong thời gian con đang chăm chú với tôi, thật đặc biệt và những trò chơi như vậy mà nghĩa là cô giáo trở nên thân thiết, thương con trong lòng cậu.
Những ngày cuối cùng vì nó đi từ những chú mèo đầu tư trước mắt, như thể làm cô giáo có được nghề ngoi trong những bức ảnh mà mẹ là một nhân vật được cô độc thưởng con mình.
Người bạn thân thiết luôn có thể nghỉ ngơi, chỉ có thể c

Epoch: 37/50... Step: 300/771... Loss: 1.2100... Val Loss: 1.0983
Thế nào có thể đọc từng điều gì đó.
Trong tiết tháo trên con đường trong các nhà thơ của em trong những chuyến đi tranh thủ viên quản ngục đã có nhiều thành tựu đẹp đẽ nhất với nhau trong những câu chuyện đều đã gắn bó với nhau với người đọc. Ngoài ra cũng đã có một luồng cảm xúc nhất của ngôi trường như nhiều câu chuyện này. Những bài văn còn nghe thấy cả ngày nữa, tôi vẫn tiếp tục theo đuổi trong các tình cảm nghiêm trọng. Nhưng tôi đã được nhiều người trong những ngày hè đến từng bước chân trong cuộc sống, những buổi sáng sớm đầy thú vị. Một chú chim đang làm tôi chuyển thành những nếp nhăn đi theo đôi tay của ba. Một lần nữa chẳng thương bố chỉ thoáng mát cả đêm nay, những cánh hoa trắng trong là những đám mây trắng và đông với những trang sắc của thời gian. Chỉ đã tiếp tục cười đùa nào cho tôi, đôi mắt đen láy như những ngôi sao mới nghe thật vui tai.
Những nụ cười mà còn là người con của bạn bè từ bao giờ trên trán

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with urllib.request.urlopen('https://raw.githubusercontent.com/hoanghuy89/rnn-lam-tho-tieng-viet/main/bai_van_mau.txt') as f:
# with open(filename+'.net', 'rb') as f:
    checkpoint = torch.load(f)
    
net = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
net.load_state_dict(checkpoint['state_dict'])
net.to(device)
net.eval() # eval mode
usr_input = input('Nhập 1 câu bất kỳ: ')
print('\n'+sample(net, 2000, top_k=4, prime=usr_input))